In [ ]:
#importing all the neccessary libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# to suppress warnings

import warnings   
warnings.filterwarnings("ignore")

# 1. Reading the Data

In [ ]:
#read & load application_data.csv: contains all the information of the client at the time of application.
#The data is about whether a client has payment difficulties.

appdata= pd.read_csv('application_data.csv')
appdata.head()

In [ ]:
#read & load previous_application.csv: contains information about the client’s previous loan data.
#It contains the data whether the previous application had been Approved, Cancelled, Refused or Unused offer.

predata= pd.read_csv('previous_application.csv')
predata.head()

# 2.A. Inspecting the Predata

In [ ]:
#Checking the number of rows and columns
predata.shape

In [ ]:
#Checking information of all the columns
predata.info(verbose=True, null_counts=True)

In [ ]:
#Checking the numeric variables of the dataframes
predata.describe()

# 2.B. Inspecting the Appdata

In [ ]:
#Checking the number of rows and columns
appdata.shape

In [ ]:
#Checking information of all the columns
appdata.info(verbose=True, null_counts=True)

In [ ]:
#Checking the numeric variables of the dataframes
appdata.describe()

# 3.A. Data Cleaning & Imputing for Predata

## 3.A.1 Null values more than 50 %

In [ ]:
# Finding the percentage of missing values in appdata dataframe
pnv = round(predata.isnull().mean()*100,2).sort_values(ascending = False)
pnv

In [ ]:
#new dataframe containing columns with null percentage > 50
pnv50 = pnv[pnv>50]
pnv50

In [ ]:
# Displaying columns having missing values more than 50%
pnv50.index

In [ ]:
#Now lets drop all the columns having missing values more than 50% that is 41 columns and storing it in a new dataframe
predata.drop(columns = pnv50.index, inplace = True)
predata

## 3.A.2 Dropping Unnesseccary columns

In [ ]:
# Listing down columns which are not needed
trash = ['WEEKDAY_APPR_PROCESS_START','HOUR_APPR_PROCESS_START','FLAG_LAST_APPL_PER_CONTRACT','NFLAG_LAST_APPL_IN_DAY']

predata.drop(trash,axis =1, inplace = True)

predata.shape

## 3.A.4 Imputing

In [ ]:
# Imputing values "Unknown" as this a categorical column
predata["NAME_TYPE_SUITE"] = predata["NAME_TYPE_SUITE"].fillna("Unknown")
round(predata.isnull().mean()*100,2).sort_values(ascending = False)

**INSIGHTS**

There are missing values in columns 'DAYS_FIRST_DUE', 'DAYS_TERMINATION', 'DAYS_FIRST_DRAWING','DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE' and these columns count days thus will keeping null values as they are

In [ ]:
# To convert negative days to postive days creating a varaible "p_days_col"
pdays = ['DAYS_DECISION','DAYS_FIRST_DRAWING', 'DAYS_FIRST_DUE', 'DAYS_LAST_DUE_1ST_VERSION', 'DAYS_LAST_DUE', 'DAYS_TERMINATION']

# Analysis before conversion
predata[pdays].describe()

In [ ]:
# Converting Negative days to positive days
predata[pdays] = abs(predata[pdays])

# analysing after conversion
predata[pdays].describe()

In [ ]:
#imputing missing values with median 
predata['AMT_ANNUITY'].fillna(predata['AMT_ANNUITY'].median(),inplace = True)
predata['AMT_ANNUITY']

In [ ]:
# Imputing null values with mode
predata['AMT_GOODS_PRICE'].fillna(predata['AMT_GOODS_PRICE'].mode()[0], inplace=True)
predata['AMT_GOODS_PRICE']

In [ ]:
#taking out values count for NAME_CONTRACT_STATUS categories where CNT_PAYMENT have null values.
predata.loc[predata['CNT_PAYMENT'].isnull(),'NAME_CONTRACT_STATUS'].value_counts()

In [ ]:
#imputing null values as 0
predata['CNT_PAYMENT'].fillna(0,inplace = True)
predata['CNT_PAYMENT'].value_counts()

# 3.B. Data Cleaning & Imputing for Appdata

## 3.B.1 Null values more than 50 %

In [ ]:
# Finding the percentage of missing values in appdata dataframe
anv = round(appdata.isnull().mean()*100,2).sort_values(ascending = False)
anv

In [ ]:
#new dataframe containing columns with null percentage > 50
anv50 = anv[anv>50]
anv50

In [ ]:
# Displaying columns having missing values more than 50%
anv50.index

In [ ]:
#Now lets drop all the columns having missing values more than 50% that is 41 columns and storing it in a new dataframe
appdata.drop(columns = anv50.index, inplace = True)
appdata

## 3.B.2 Null values more than 14 %

In [ ]:
nv = round(appdata.isnull().mean()*100,2).sort_values(ascending = False)
nv

In [ ]:
nv15 = nv[nv>15]
nv15

In [ ]:
# removing 'OCCUPATION_TYPE', 'EXT_SOURCE_3' so that we can drop all other at once.
nv15.drop(["OCCUPATION_TYPE","EXT_SOURCE_3"], inplace = True)
nv15

In [ ]:
# columns having missing values more than 15% and are not reletable
nv15.index

In [ ]:
# droping the columns having missing values more than 15% and are not reletable
appdata.drop(nv15.index,axis=1, inplace = True)
appdata

In [ ]:
# Checking the % of null values for each column in new dataset
null = appdata.isnull().sum()/len(appdata)*100
null.sort_values(ascending = False).head(60)

## 3.B.3 Dropping Unnesseccary columns

In [ ]:
#list of columns to be dropped as these columns have no significant value for the analysis
flag_columns = [col for col in appdata.columns if "FLAG" in col]
flag_columns

In [ ]:
appdata.drop(flag_columns, axis = 1, inplace= True)
appdata.shape

## 3.B.4 Imputing

In [ ]:
print(appdata.CODE_GENDER.value_counts())
# Replacing XNA value with F
appdata.loc[appdata.CODE_GENDER == 'XNA','CODE_GENDER'] = 'F'
appdata.CODE_GENDER.value_counts()

In [ ]:
print(appdata.OCCUPATION_TYPE.value_counts())
# Replacing null values with "Unknown"
appdata["OCCUPATION_TYPE"] = appdata["OCCUPATION_TYPE"].fillna("Unknown")
appdata.OCCUPATION_TYPE.value_counts()

In [ ]:
# Plotting a percentage graph having each category of "OCCUPATION_TYPE"
plt.figure(figsize = [12,7])
(appdata["OCCUPATION_TYPE"].value_counts()).plot.bar(width = .7)
plt.title("Percentage of Type of Occupations", fontdict={"fontsize":20}, pad =20,color = "#4472C4")
plt.show()

**INSIGHTS**

Highest % of values belong to Unknown category and Laborers has the Second Highest.
Lowest being the IT staff.

In [ ]:
print(appdata.ORGANIZATION_TYPE.value_counts())
# Replacing XNA with NaN
appdata = appdata.replace('XNA',np.NaN)
appdata.ORGANIZATION_TYPE.value_counts()

In [ ]:
# Plotting a percentage graph having each category of "ORGANIZATION_TYPE"
plt.figure(figsize = [12,7])
(appdata["ORGANIZATION_TYPE"].value_counts()).plot.bar(width = .7)
plt.title("Percentage of Type of Occupations", fontdict={"fontsize":20}, pad =20,color = "#4472C4")
plt.show()

In [ ]:
#creating "amt_credit" variable having these columns "AMT_REQ_CREDIT_BUREAU_YEAR","AMT_REQ_CREDIT_BUREAU_QRT","AMT_REQ_CREDIT_BUREAU_MON","AMT_REQ_CREDIT_BUREAU_WEEK",
#"AMT_REQ_CREDIT_BUREAU_DAY","AMT_REQ_CREDIT_BUREAU_HOUR"
amt_credit = ["AMT_REQ_CREDIT_BUREAU_YEAR","AMT_REQ_CREDIT_BUREAU_QRT","AMT_REQ_CREDIT_BUREAU_MON","AMT_REQ_CREDIT_BUREAU_WEEK",
"AMT_REQ_CREDIT_BUREAU_DAY","AMT_REQ_CREDIT_BUREAU_HOUR"]

#filling missing values with median values
appdata.fillna(appdata[amt_credit].median(),inplace = True)

appdata

# 4.A. Binning Variables for Predata

In [ ]:
#days group calculation e.g. 369 will be grouped as with in 2 years
bins = [0,1*365,2*365,3*365,4*365,5*365,6*365,7*365,10*365]
slots = ["1","2","3","4","5","6","7","7 above"]
predata['YEARLY_DECISION'] = pd.cut(predata['DAYS_DECISION'],bins,labels=slots)

predata['YEARLY_DECISION'].value_counts(normalize=True)*100

**INSIGHTS**

Almost 35% loan applicatants have applied for a new loan within 1 year of previous loan decision


In [ ]:
#Converting required categoical columns from Object to categorical 

pcatgorical = ['NAME_CASH_LOAN_PURPOSE','NAME_CONTRACT_STATUS','NAME_PAYMENT_TYPE','CODE_REJECT_REASON',
               'NAME_CLIENT_TYPE','NAME_GOODS_CATEGORY','NAME_PORTFOLIO','NAME_PRODUCT_TYPE','CHANNEL_TYPE',
               'NAME_SELLER_INDUSTRY','NAME_YIELD_GROUP','PRODUCT_COMBINATION', 'NAME_CONTRACT_TYPE']
for col in pcatgorical:
    predata[col] =pd.Categorical(predata[col])

len(pcatgorical)

# 4.B. Binning Variables for Appdata

In [ ]:
# Creating bins for income amount in term of Lakhs
appdata['AMT_INCOME_TOTAL']=appdata['AMT_INCOME_TOTAL']/100000

bins = [0,1,2,3,4,5,6,7,8,9,10,11]
slot = ['0-1L','1L-2L', '2L-3L','3L-4L','4L-5L','5L-6L','6L-7L','7L-8L','8L-9L','9L-10L','10L Above']

appdata['AMT_INCOME_RANGE']=pd.cut(appdata['AMT_INCOME_TOTAL'],bins,labels=slot)
round((appdata["AMT_INCOME_RANGE"].value_counts(normalize = True)*100),2)

In [ ]:
# Creating bins for Credit amount in term of Lakhs
appdata['AMT_CREDIT']=appdata['AMT_CREDIT']/100000

bins = [0,1,2,3,4,5,6,7,8,9,10,100]
slots = ['0-1L','1L-2L', '2L-3L','3L-4L','4L-5L','5L-6L','6L-7L','7L-8L','8L-9L','9L-10L','10L Above']

appdata['AMT_CREDIT_RANGE']=pd.cut(appdata['AMT_CREDIT'],bins=bins,labels=slots)
round((appdata["AMT_CREDIT_RANGE"].value_counts(normalize = True)*100),2)

In [ ]:
# Creating bins for Price of Goods in term of Lakhs
appdata['AMT_GOODS_PRICE']=appdata['AMT_GOODS_PRICE']/100000

bins = [0,1,2,3,4,5,6,7,8,9,10,100]
slots = ['0-1L','1L-2L', '2L-3L','3L-4L','4L-5L','5L-6L','6L-7L','7L-8L','8L-9L','9L-10L','10L Above']

appdata['AMT_GOODS_PRICE_RANGE']=pd.cut(appdata['AMT_GOODS_PRICE'],bins=bins,labels=slots)
round((appdata["AMT_GOODS_PRICE_RANGE"].value_counts(normalize = True)*100),2)

In [ ]:
# creating "days_col" varibale to store all days columns
dayscol = ["DAYS_BIRTH", "DAYS_EMPLOYED", "DAYS_REGISTRATION", "DAYS_ID_PUBLISH", "DAYS_LAST_PHONE_CHANGE"]
appdata[dayscol].describe()

In [ ]:
#using abs() function to correct the days values
appdata[dayscol]= abs(appdata[dayscol])
appdata[dayscol].describe()

In [ ]:
# Convert DAYS_BIRTH, DAYS_EMPLOYED columns in terms of Years and binning years for better understanding,
# that is adding two more categorical column
appdata["AGE"] = appdata["DAYS_BIRTH"]/365
bins = [0,20,25,30,35,40,45,50,55,60,100]
slots = ["0-20","20-25","25-30","30-35","35-40","40-45","45-50","50-55","55-60","60 Above"]

appdata["AGE_GROUP"] = pd.cut(appdata["AGE"], bins=bins, labels=slots)
appdata["AGE_GROUP"].value_counts(normalize= True)*100

In [ ]:
# Converting 'DAYS_BIRTH' to years
appdata['DAYS_BIRTH']= (appdata['DAYS_BIRTH']/365).astype(int)
bins=[19,25,35,60,100]
slots=['Very_Young','Young', 'Middle_Age', 'Senior_Citizen']

appdata['DAYS_BIRTH_BINS']=pd.cut(appdata['DAYS_BIRTH'], bins=bins, labels=slots)
appdata['DAYS_BIRTH_BINS'].value_counts()

In [ ]:
#creating column "EMPLOYEMENT_YEARS" from "DAYS_EMPLOYED"
appdata["YEARS_EMPLOYED"] = appdata["DAYS_EMPLOYED"]/365
bins = [0,5,10,15,20,25,30,50]
slots = ["0-5","5-10","10-15","15-20","20-25","25-30","30 Above"]

appdata["EMPLOYEMENT_YEARS"] = pd.cut(appdata["YEARS_EMPLOYED"], bins=bins, labels=slots)
appdata["EMPLOYEMENT_YEARS"].value_counts(normalize= True)*100

In [ ]:
#Converting Desired columns from Object to Categorical column
#from the list, we have taken out the desired columns for conversion
categorical = ['NAME_CONTRACT_TYPE','CODE_GENDER','NAME_TYPE_SUITE','NAME_INCOME_TYPE','NAME_EDUCATION_TYPE',
               'NAME_FAMILY_STATUS','NAME_HOUSING_TYPE','OCCUPATION_TYPE','WEEKDAY_APPR_PROCESS_START', 'ORGANIZATION_TYPE',
               'LIVE_CITY_NOT_WORK_CITY','REG_CITY_NOT_LIVE_CITY','REG_CITY_NOT_WORK_CITY','REG_REGION_NOT_WORK_REGION',
               'LIVE_REGION_NOT_WORK_REGION','REGION_RATING_CLIENT','WEEKDAY_APPR_PROCESS_START','REGION_RATING_CLIENT_W_CITY','CNT_FAM_MEMBERS']

for col in categorical:
    appdata[col] = pd.Categorical(appdata[col])
    
# Converting total of 21 columns to categorical one
len(categorical)

# 5.A. Identifying Outliers for Predata

In [ ]:
poutlier = ['AMT_ANNUITY','AMT_APPLICATION','AMT_CREDIT','AMT_GOODS_PRICE','SELLERPLACE_AREA','DAYS_DECISION','CNT_PAYMENT']
plt.figure(figsize=[15,30])
for i,j in enumerate(poutlier):
    plt.subplot(3,3,i+1)
    sns.boxplot(y = predata[j],color = "#70AD47")
    plt.yticks(fontsize=8)
    plt.xlabel(poutlier[i])
    plt.ylabel("")
    plt.title(i)
    plt.savefig("poutliers.png")

**INSIGHTS**

It can be seen that in previous application data

- AMT_ANNUITY, AMT_APPLICATION, AMT_CREDIT, AMT_GOODS_PRICE, SELLERPLACE_AREA have huge number of outliers.
- CNT_PAYMENT has few outlier values.
- DAYS_DECISION has little number of outliers indicating that these previous applications decisions were taken long back.

In [ ]:
#orginial shape of the appdata before removing outliers
pog = predata.shape
pog

In [ ]:
# removing outliers using 1.5 IQR method and storing it in a new dataframe
ptemporary=predata[:]
for out in poutlier: 
    q1 = ptemporary[out].quantile(0.25)
    q3 = ptemporary[out].quantile(0.75)
    iqr = q3-q1
    range_low  = q1-(1.5*iqr)
    range_high = q3+(1.5*iqr)
    ptemp = ptemporary[(ptemporary[out] < range_low) | (ptemporary[out] > range_high)]
    ptemporary.drop(ptemp.index , inplace=True)

In [ ]:
# percentage data lost 
100 * ((pog[0]-ptemporary.shape[0] ) / pog[0])

# 5.B. Identifying Outliers for Appdata

In [ ]:
outlier = ["CNT_CHILDREN","AMT_INCOME_TOTAL", "AMT_CREDIT", "AMT_ANNUITY", "AMT_GOODS_PRICE", "DAYS_BIRTH", "DAYS_EMPLOYED", "DAYS_REGISTRATION"]
plt.figure(figsize=[15,30])
for i,j in enumerate(outlier):
    plt.subplot(3,3,i+1)
    sns.boxplot(y = appdata[j],color = "#4472C4")
    plt.yticks(fontsize=8)
    plt.xlabel(outlier[i])
    plt.ylabel("")
    plt.title(i)
    plt.savefig("outliers.png")

**INSIGHTS**

It can be seen that in current application data
- AMT_ANNUITY, AMT_CREDIT, AMT_GOODS_PRICE,CNT_CHILDREN have some number of outliers.
- AMT_INCOME_TOTAL has huge number of outliers which indicate that few of the loan applicants have high income when compared to the others.
- DAYS_BIRTH has no outliers which means the data available is reliable.
- DAYS_EMPLOYED has outlier values around 350000(days) which is around 958 years which is impossible and hence this has to be incorrect entry.

In [ ]:
#orginial shape of the appdata before removing outliers
og = appdata.shape
og

In [ ]:
# removing outliers using 1.5 IQR method and stroing it in a new dataframe
temporary=appdata[:]
for out in outlier: 
    q1 = temporary[out].quantile(0.25)
    q3 = temporary[out].quantile(0.75)
    iqr = q3-q1
    range_low  = q1-(1.5*iqr)
    range_high = q3+(1.5*iqr)
    temp = temporary[(temporary[out] < range_low) | (temporary[out] > range_high)]
    temporary.drop(temp.index , inplace=True)

In [ ]:
# percentage data lost 
100 * ((og[0]- temporary.shape[0] ) / og[0])

# 6. Imbalance Data

In [ ]:
temp1=appdata["TARGET"].value_counts()
plt.figure(figsize= [14,6])
patches, texts, pcts=plt.pie(appdata["TARGET"].value_counts(),labels=["Repayer","Defaulter"],colors=["#70AD47","#4472C4"],autopct='%.0f%%',wedgeprops={'linewidth': 2.0, 'edgecolor': 'white'},textprops={'size': '14'})
plt.setp(pcts, color='white', fontweight='bold',fontsize=14)
plt.title("Imbalance Plotting (Repayer Vs Defaulter)", fontdict = {"fontsize":18})
plt.show()

Imbalance Ratio with respect to Repayer and Defaulter is given: **11.39/1 (approx)**

# 7. Exploratory Data Analysis

## 7.1 Univariate Analysis

In [ ]:
# Dividing the original dataset into two different datasets depending upon the target value
tgt0 = appdata.loc[appdata.TARGET == 0]
tgt1 = appdata.loc[appdata.TARGET == 1]
count1=0

# Function for univariate plots
def punivariate(tg,title,col,hue =None):
    label1=tg[col].value_counts().index
    plt.figure(figsize= [20,5])
    patches, texts, pcts=plt.pie(tg[col].value_counts(),autopct='%.0f%%',wedgeprops={'linewidth': 2.0, 'edgecolor': 'white'},textprops={'size': '14'})
    plt.setp(pcts, color='white', fontweight='bold',fontsize=14)
    plt.title(title, fontdict = {"fontsize":18})
    plt.legend(label1,loc='upper left', bbox_to_anchor =(1,1,0.5,0))
    plt.tight_layout()
    plt.show()

In [ ]:
# Income range of Loan-Non Payment Difficulties
punivariate(tg=tgt0, title='Income Range of Loan- Non Payment Difficulties',col="AMT_INCOME_RANGE")

In [ ]:
# Income range of Loan Payment Difficulties
punivariate(tg=tgt1,title='Income Range of Loan Payment Difficulties',col="AMT_INCOME_RANGE")

In [ ]:
# Education of Loan-Non Payment Difficulties
punivariate(tg=tgt0, title='Education of Loan- Non Payment Difficulties',col="NAME_EDUCATION_TYPE")

In [ ]:
# Income range of Loan Payment Difficulties
punivariate(tg=tgt1,title='Education of Loan Payment Difficulties',col="NAME_EDUCATION_TYPE")

## 7.2 Segmented Univariate Analysis

In [ ]:
# Dividing the original dataset into two different datasets depending upon the target value
tgt0 = appdata.loc[appdata.TARGET == 0]
tgt1 = appdata.loc[appdata.TARGET == 1]
count=0

# Function for univariate plots
def univariate(df,col,hue =None):
    sns.set(style="darkgrid")
    fig, ax=plt.subplots(nrows =1,ncols=2,figsize=(18,8))
    ax[0].set_title("Distribution Plot",fontdict = {"fontsize":18})
    sns.distplot(df[~df[col].isna()][col],ax=ax[0], color="#4472C4")
    ax[1].set_title("Box Plot",fontdict = {"fontsize":18})
    sns.boxplot(data=df, x=col,ax=ax[1],orient='.', color="#4472C4")
    plt.show()
    
# Distribution and Box plot for 'AMT_ANNUITY' for Loan Non-Payment Difficulties
univariate(df=tgt0,col='AMT_ANNUITY')

In [ ]:
# Distribution and Box plot for 'AMT_ANNUITY' for Loan- Payment Difficulties
univariate(df=tgt1,col='AMT_ANNUITY')

In [ ]:
# Distribution and Box plot for 'AMT_CREDIT' for Loan Non-Payment Difficulties
univariate(df=tgt0,col='AMT_CREDIT')

In [ ]:
# Distribution and Box plot for 'AMT_CREDIT' for Loan Payment Difficulties
univariate(df=tgt1,col='AMT_CREDIT')

In [ ]:
# Distribution and Box plot for 'AMT_GOODS_PRICE' for Loan- Non-Payment Difficulties
univariate(df=tgt0,col='AMT_GOODS_PRICE')

In [ ]:
# Distribution and Box plot for 'AMT_GOODS_PRICE' for Loan-Payment Difficulties
univariate(df=tgt1,col='AMT_GOODS_PRICE')

**INSIGHTS**

- Most no. of loans are given for goods price below 10 lakhs
- Credit amount of the loan is mostly less then 10 lakhs
- Most people pay annuity below 50K for the credit loan

## 7.3 Bivariate Analysis

In [ ]:
appdata.groupby('NAME_INCOME_TYPE')['AMT_INCOME_TOTAL'].describe()

In [ ]:
# function for plotting repetitive barplots in bivariate categorical analysis
def bivariate_categorical(x,y,df,hue,figsize,labels):
    plt.figure(figsize=figsize)
    sns.barplot(x=x,y=y,data=df, hue=hue,palette=('#70AD47','#4472C4'))     
        
    # Defining aesthetics of Labels and Title of the plot using style dictionaries
    plt.xlabel(x,fontsize = 15)    
    plt.ylabel(y,fontsize = 15)   
    plt.xticks(rotation=45, ha='right')
    plt.legend(labels = labels )
    plt.show()

In [ ]:
# Income type vs Income Amount Range on a Seaborn Barplot
bivariate_categorical("NAME_INCOME_TYPE","AMT_INCOME_TOTAL",appdata,"TARGET",(15,6),['Repayer','Defaulter'])

**INSIGHTS**

It can be seen that Businessman income is the highest and the estimated range seem to indicate that the income of a Businessman could be in the range close to 4 lakhs and slightly above 10 lakhs.

## 7.4 Bivariate Analysis of Categorical vs Numerical Variables

In [ ]:
# 'NAME_EDUCATION_TYPE' vs 'AMT_CREDIT' for Loan Non-Payment Difficulties
fig = px.box(tgt0, x="NAME_EDUCATION_TYPE", y="AMT_CREDIT", color='NAME_FAMILY_STATUS',
                title="Credit amount vs Education of Loan Non-Payment Difficulties")

fig.show()

In [ ]:
# 'NAME_EDUCATION_TYPE' vs 'AMT_CREDIT' for Loan Payment Difficulties
fig = px.box(tgt1, x="NAME_EDUCATION_TYPE", y="AMT_CREDIT", color='NAME_FAMILY_STATUS',
                title="Credit amount vs Education of Loan Payment Difficulties")

fig.show()

**INSIGHTS** 

The graphs *Credit Amount VS Education* for Loan Payment Difficulties and Loan Non-Payment Difficulties appears to be similar.

- We observe that Family status of 'civil marriage', 'marriage' and 'separated' of Academic degree education are having higher number of credits than others.
- Most of the outliers are from Education type 'Higher education' and 'Secondary'.
- Civil marriage for Academic degree is having most of the credits in the 3rd Quartile.

In [ ]:
# 'AMT_INCOME_RANGE' vs 'AMT_CREDIT' for Loan - Non Payment Difficulties
fig = px.box(tgt0, x="AMT_INCOME_RANGE", y="AMT_CREDIT", color='NAME_FAMILY_STATUS',
                title="Income range vs Credit amount of Loan Non- Payment Difficulties")

fig.show()

In [ ]:
# 'AMT_INCOME_RANGE' vs 'AMT_CREDIT' for Loan Payment Difficulties
fig = px.box(tgt1, x="AMT_INCOME_RANGE", y="AMT_CREDIT", color='NAME_FAMILY_STATUS',
                title="Income range vs Credit amount of Loan Payment Difficulties")

fig.show()

**INSIGHTS** 

The graphs *Credit Amount VS Income Range* for Loan Payment Difficulties and Loan Non-Payment Difficulties appears to be similar.

- We observe that ‘Single/ not Married’ have the highest credit with income range of 9L – 10L
- Maximum outliers are for ‘Married’ Applicant in the range of income 1L – 2L
- ‘Single’ Applicants with Loan Payment Difficulties with Credit > 20 have income range of 9L to 10L in 3rd Quartile
- ‘Separated’ Applicants with Loan Non-Payment Difficulties with Credit > 25 have income range of 10L & Above in 3rd Quartile

## 7.5 Numeric Variables Analysis

In [ ]:
#Pairplot for Target 0 (Loan Non-Payment Difficulties)
pair0 = tgt0[['AMT_CREDIT', 'AMT_ANNUITY', 'AMT_INCOME_TOTAL', 'AMT_GOODS_PRICE', 'DAYS_BIRTH']].fillna(0)
sns.pairplot(pair0)
plt.show()

In [ ]:
#Pairplot for Target 1 (Loan Payment Difficulties)
pair1 = tgt1[['AMT_CREDIT', 'AMT_ANNUITY', 'AMT_INCOME_TOTAL', 'AMT_GOODS_PRICE', 'DAYS_BIRTH']].fillna(0)
sns.pairplot(pair1)
plt.show()

In [ ]:
#Listing all the columnns of dataframe "appl_data"
appdata.columns

# bisecting the app_data dataframe based on Target value 0 and 1 for correlation and other analysis
colms_correlation = ['NAME_CONTRACT_TYPE', 'CODE_GENDER',
                        'CNT_CHILDREN', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'AMT_GOODS_PRICE', 
                        'NAME_TYPE_SUITE', 'NAME_INCOME_TYPE', 'NAME_EDUCATION_TYPE', 'NAME_FAMILY_STATUS',
                        'NAME_HOUSING_TYPE', 'REGION_POPULATION_RELATIVE', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 
                        'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH', 'OCCUPATION_TYPE', 'CNT_FAM_MEMBERS', 'REGION_RATING_CLIENT',
                        'REGION_RATING_CLIENT_W_CITY', 'WEEKDAY_APPR_PROCESS_START', 'HOUR_APPR_PROCESS_START',
                        'REG_REGION_NOT_LIVE_REGION', 'REG_REGION_NOT_WORK_REGION', 'LIVE_REGION_NOT_WORK_REGION', 
                        'REG_CITY_NOT_LIVE_CITY', 'REG_CITY_NOT_WORK_CITY', 'LIVE_CITY_NOT_WORK_CITY', 'ORGANIZATION_TYPE',
                        'OBS_60_CNT_SOCIAL_CIRCLE', 'DEF_60_CNT_SOCIAL_CIRCLE', 'DAYS_LAST_PHONE_CHANGE',
                        'AMT_REQ_CREDIT_BUREAU_HOUR', 'AMT_REQ_CREDIT_BUREAU_DAY', 'AMT_REQ_CREDIT_BUREAU_WEEK',
                        'AMT_REQ_CREDIT_BUREAU_MON', 'AMT_REQ_CREDIT_BUREAU_QRT', 'AMT_REQ_CREDIT_BUREAU_YEAR']

# Repayers dataframe
repayer_dataframe = appdata.loc[appdata['TARGET']==0, colms_correlation]
repayer_dataframe.head()

In [ ]:
# Defaulters dataframe
defaulter_dataframe = appdata.loc[appdata['TARGET']==1, colms_correlation]
defaulter_dataframe.head()

In [ ]:
# Getting  top 10 correlation for the Repayers dataframe

repayer_corr = repayer_dataframe.corr()
corr_df_repayer = repayer_corr.where(np.triu(np.ones(repayer_corr.shape),k=1).astype(np.bool)).unstack().reset_index()
corr_df_repayer.columns =['VAR1','VAR2','Correlation']
corr_df_repayer.dropna(subset = ["Correlation"], inplace = True)
corr_df_repayer["Correlation"]=corr_df_repayer["Correlation"].abs() 
corr_df_repayer.sort_values(by='Correlation', ascending=False, inplace=True) 
corr_df_repayer.head(10)

In [ ]:
#plotting heatmap to see linear correlation amoung Repayers 
fig = plt.figure(figsize=(20,18))
ax = sns.heatmap(repayer_dataframe.corr(), cmap="mako",annot=True,linewidth =1)

In [ ]:
# Getting the top 10 correlation for the Defaulter data
corr_Defaulter = defaulter_dataframe.corr()
corr_Defaulter = corr_Defaulter.where(np.triu(np.ones(corr_Defaulter.shape),k=1).astype(np.bool))
corr_df_Defaulter = corr_Defaulter.unstack().reset_index()
corr_df_Defaulter.columns =['VAR1','VAR2','Correlation']
corr_df_Defaulter.dropna(subset = ["Correlation"], inplace = True)
corr_df_Defaulter["Correlation"]=corr_df_Defaulter["Correlation"].abs()
corr_df_Defaulter.sort_values(by='Correlation', ascending=False, inplace=True)
corr_df_Defaulter.head(10)

In [ ]:
#plotting heatmap to see linear correlation amoung Defaulter
fig = plt.figure(figsize=(20,15))
ax = sns.heatmap(defaulter_dataframe.corr(), cmap="mako",annot=True,linewidth =1)

**INSIGHTS** 

1. Credit amount is highly correlated with good price amount which is same as repayers.

2. Loan annuity correlation with credit amount has slightly reduced in defaulters(0.75) when compared to repayers(0.77).

3. We can also see that repayers have high correlation in number of days employed(0.62) when compared to defaulters(0.58).

4. There is a slight increase in defaulted to observed count in social circle among defaulters(0.264) when compared to repayers(0.254)

## 7.6 Merged Dataframes Analysis

In [ ]:
# merge both the dataframe on SK_ID_CURR with Inner Joins
loanapp = pd.merge(appdata, predata, how='inner', on='SK_ID_CURR')
loanapp.head()

In [ ]:
#Checking the details of the merged dataframe
loanapp.shape

In [ ]:
# checking the columns and column types of the dataframe
loanapp.info()

In [ ]:
#function for plotting repetitive countplots in univariate categorical analysis on the merged data frame
def univariate_category_merged(col,df,hue,palette,ylog,figsize):
    plt.figure(figsize=figsize)
    ax=sns.countplot(x=col, data=df,hue= hue,palette= palette,order=df[col].value_counts().index)
    
    if ylog:
        plt.yscale('log')
        plt.ylabel("Count (log)",fontsize=15)     
    else:
        plt.ylabel("Count",fontsize=15)       

    plt.title((col+" VS "+hue) , fontsize=20) 
    plt.legend(loc = "upper right")
    plt.xticks(rotation=45, ha='right')
    
    plt.show()

In [ ]:
# Bisecting the "loan_df" dataframe based on Target value 0 and 1 for correlation and other analysis

loanapp0 = loanapp[loanapp['TARGET']==0] # Repayers
loanapp1 = loanapp[loanapp['TARGET']==1] # Defaulters

In [ ]:
#NAME_CASH_LOAN_PURPOSE vs NAME_CONTRACT_STATUS for Loan Non-Payment Difficulties
univariate_category_merged("NAME_CASH_LOAN_PURPOSE",loanapp0,"NAME_CONTRACT_STATUS",'mako',True,(18,7))

In [ ]:
#NAME_CASH_LOAN_PURPOSE vs NAME_CONTRACT_STATUS for Loan Payment Difficulties
univariate_category_merged("NAME_CASH_LOAN_PURPOSE",loanapp1,"NAME_CONTRACT_STATUS",'mako',True,(18,7))

**INSIGHTS**

1. Loan purpose has high number of unknown values (XAP, XNA)
2. Loan taken for the purpose of Repairs looks to have highest default rate
3. Huge number application have been rejected by bank or refused by client which are applied for Repair or Other. from this we can infer that repair is considered high risk by bank. Also, either they are rejected or bank offers loan on high interest rate which is not feasible by the clients and they refuse the loan.

In [ ]:
# Checking Contract Status based on loan repayment status whether there is any business loss or financial loss
univariate_category_merged("NAME_CONTRACT_STATUS",loanapp,"TARGET",'mako',False,(14,8))
l = loanapp.groupby("NAME_CONTRACT_STATUS")["TARGET"]
df = pd.concat([l.value_counts(),round(l.value_counts(normalize=True).mul(100),2)],axis=1, keys=('Counts','Percentage'))
df['Percentage'] = df['Percentage'].astype(str) +"%" # adding percentage symbol in the results for understanding
df

**INSIGHTS**

1. 90% of the previously cancelled client have actually repayed the loan. Revising the interest rates would increase business      opportunity for these clients
2. 88% of the clients who have been previously refused a loan has payed back the loan in current case.

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
def biplot(df,feature,title):
    temp = df[feature].value_counts()
    
    # Calculate the percentage of target=1 per category value
    perc = df[[feature, 'TARGET']].groupby([feature],as_index=False).mean()
    perc.sort_values(by='TARGET', ascending=False, inplace=True)

    fig = make_subplots(rows=1, cols=2,subplot_titles=("Count of "+ title,"% of Loan Payment difficulties within each category"))

    fig.add_trace(go.Bar(x=temp.index, y=temp.values),row=1, col=1)
    fig.add_trace(go.Bar(x=perc[feature].to_list(), y=perc['TARGET'].to_list()),row=1, col=2)
    fig['layout']['xaxis']['title']=feature
    fig['layout']['xaxis2']['title']=feature
    fig['layout']['yaxis']['title']='Count'
    fig['layout']['yaxis2']['title']='% of Loan Payment Difficulties'
    fig.update_layout(height=600, width=1000, title_text=title, showlegend=False)
    fig.show()

In [ ]:
# Distribution of Contract Status and its category with maximum % of Loan-Payment Difficulties
biplot(loanapp,'NAME_CONTRACT_STATUS', 'CONTRACT STATUS')

**INSIGHTS**

From the first graph it can be seen that most of the contract type from previous application was 'Cash loans'
It can be clearly seen from the second graph that the

1. 'Revolving Loans' contracts from previous application are the ones who have maximum % of Loan-Payment Difficulties from current application.
2. 'Consumer loans' contracts from previous application are the ones who have minimum % of Loan-Payment Difficulties from current application.

In [ ]:
# Distribution of Payment Type and its category with maximum % of Loan-Payment Difficulties
biplot(loanapp,'NAME_PAYMENT_TYPE', 'PAYMENT TYPE')

**INSIGHTS**

From the first graph it can be seen that most of the payment type from previous application was 'Cash through bank'.
It can be clearly seen from the second graph that all three types of payments from the previous application have almost same % of Loan-Payment Difficulties from current application

In [ ]:
# Distribution of Cash Loan Purpose and its category with maximum % of Loan-Payment Difficulties 
biplot(loanapp,'NAME_CASH_LOAN_PURPOSE', 'CASH LOAN PURPOSE')

**INSIGHTS**

From the first graph it can be seen that purpose of cash loan from previous data was maximum for 'Repairs'

It can be clearly seen from the second graph that the 'Refusal to name the goal' for cash loan from previous application are the ones who have maximum % of Loan-Payment Difficulties from current application.

In [ ]:
# Distribution of Client Type and its category with maximum % of Loan-Payment Difficulties 
biplot(loanapp,'NAME_CLIENT_TYPE', 'CLIENT TYPE')

**INSIGHT**

From the first graph it can be seen that most of the clients from previous application are 'Repeater'

It can be clearly seen from the second graph that the

1. 'New' clients from previous application are the ones who have maximum % of Loan-Payment Difficulties from current application.
2. 'Refreshed' clients from previous application are the ones who have minimum % of Loan-Payment Difficulties from current application.

In [ ]:
table1 = pd.pivot_table(loanapp, values='TARGET', index=['NAME_CLIENT_TYPE'],columns=['NAME_CONTRACT_STATUS'], aggfunc=np.mean)
table1

In [ ]:
plt.figure(figsize= (21,10))
ax = table1.T.plot(kind='bar')
ylab = ax.set_ylabel('% of Loan-Payment Difficulties')
plt.title('% of Loan Payment Difficulties for NAME_CONTRACT_STATUS and NAME_CLIENT_TYPE', fontdict={'fontsize':15})
plt.legend(loc='upper left', bbox_to_anchor =(1,1,0.5,0))
plt.show()

**INSIGHTS**

It can be observed from the above graph that Client who where 'New' and had 'Cancelled' previous application tend to have more % of Loan-Payment Difficulties in current application

In [ ]:
table2 = pd.pivot_table(loanapp, values='TARGET', index=['NAME_CONTRACT_TYPE_y'],columns=['NAME_CONTRACT_STATUS'], aggfunc=np.mean)
table2

In [ ]:
plt.figure(figsize= (10,8))
ax = table2.T.plot(kind='bar')
ylab = ax.set_ylabel('% of Loan-Payment Difficulties')
plt.title('% of Loan Payment Difficulties for NAME_CONTRACT_STATUS and NAME_CONTRACT_TYPE', fontdict={'fontsize':15})
plt.legend(loc='upper left', bbox_to_anchor =(1,1,0.5,0))
plt.show()

**INSIGHTS**

It can be observed from the above graph that maximum loans were Refused and lesser loans were approved. It is observed that ‘Revolving Loans’ have the highest average % of loan-payment difficulties.

In [ ]:
# Function to plot point plots 
def pointplot(df,hue,x,y):
    plt.figure(figsize=(12,6))
    sns.pointplot(x=x, y=y, hue=hue, data=df,palette='mako')
    plt.title(x+" VS "+y,fontsize = 15)

In [ ]:
# plotting the relationship between income total and contact status
pointplot(loanapp,"TARGET","NAME_CONTRACT_STATUS",'AMT_INCOME_TOTAL')

In [ ]:
# plotting the relationship between people who defaulted in last 60 days being in client's social circle and contact status
pointplot(loanapp,"TARGET","NAME_CONTRACT_STATUS",'DEF_60_CNT_SOCIAL_CIRCLE')

# Conclusion

After analysing the datasets, there are few attributes of a client with which the bank would be able to identify if they will repay the loan or not. The analysis is consised as below with the contributing factors and categorization:

### A. Decisive Factor whether an applicant will be Repayer:

1. NAME_EDUCATION_TYPE: Academic degree has less defaults.
2. NAME_INCOME_TYPE: Student and Businessmen have no defaults.
3. REGION_RATING_CLIENT: RATING 1 is safer.
4. ORGANIZATION_TYPE: Clients with Trade Type 4 and 5 and Industry type 8 have defaulted less than 3%
5. DAYS_BIRTH: People above age of 50 have low probability of defaulting
6. DAYS_EMPLOYED: Clients with 40+ year experience having less than 1% default rate
7. AMT_INCOME_TOTAL:Applicant with Income more than 700,000 are less likely to default
8. NAME_CASH_LOAN_PURPOSE: Loans bought for Hobby, Buying garage are being repayed mostly.
9. CNT_CHILDREN: People with zero to two children tend to repay the loans.

### B. Decisive Factor whether an applicant will be Defaulter:
    
1. CODE_GENDER: Men are at relatively higher default rate
2. NAME_FAMILY_STATUS : People who have civil marriage or who are single default a lot.
3. NAME_EDUCATION_TYPE: People with Lower Secondary and Secondary education
4. NAME_INCOME_TYPE: Clients who are either at Maternity leave OR Unemployed default a lot.
5. REGION_RATING_CLIENT: People who live in Rating 3 has highest defaults.
6. OCCUPATION_TYPE: Avoid Low-skill Laborers, Drivers and Waiters/barmen staff, Security staff, Laborers and Cooking staff as their default rate is huge.
7. ORGANIZATION_TYPE: Organizations with highest percent of loans not repaid are Transport: type 3 (16%), Industry: type 13 (13.5%), Industry: type 8 (12.5%) and Restaurant (less than 12%). Self-employed people have relative high defaulting rate, and thus should be avoided to be approved for loan or provide loan with higher interest rate to mitigate the risk of defaulting.
8. DAYS_BIRTH: Avoid young people who are in age group of 20-40 as they have higher probability of defaulting
9. DAYS_EMPLOYED: People who have less than 5 years of employment have high default rate.
10. CNT_CHILDREN & CNT_FAM_MEMBERS: Client who have children equal to or more than 9 default 100% and hence their applications are to be rejected.
11. AMT_GOODS_PRICE: When the credit amount goes beyond 3lakhs, there is an increase in defaulters.

### C. Factors that Loan can be given on Condition of High Interest rate to mitigate any default risk leading to business loss:

1. NAME_HOUSING_TYPE: High number of loan applications are from the category of people who live in Rented apartments & living with parents and hence offering the loan would mitigate the loss if any of those default.
2. AMT_CREDIT: People who get loan for 3-6 Lakhs tend to default more than others and hence having higher interest specifically for this credit range would be ideal.
3. AMT_INCOME: Since 90% of the applications have Income total less than 3Lakhs and they have high probability of defaulting, they could be offered loan with higher interest compared to other income category.
4. CNT_CHILDREN & CNT_FAM_MEMBERS: Clients who have 4 to 8 children has a very high default rate and hence higher interest should be imposed on their loans.
5. NAME_CASH_LOAN_PURPOSE: Loan taken for the purpose of Repairs seems to have highest default rate. A very high number applications have been rejected by bank or refused by client in previous applications as well which has purpose as repair or other. This shows that purpose repair is taken as high risk by bank and either they are rejected, or bank offers very high loan interest rate which is not feasible by the clients, thus they refuse the loan. The same approach could be followed in future as well.